### Yelp Dataset 10Mar2025 Validations or Proof of Concepts
#### 1. Extrat Tar File

In [1]:
# !pip3 install -r requirements.txt

The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.


In [4]:
from src import modules as f
%load_ext autoreload
%autoreload 2

In [5]:
# load for the first time to set up files
# f.json_2_parquet()

In [6]:
# Check for spark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("MyRapidsSparkApp") \
    .master("local[*]") \
    .config("spark.jars", "/home/oem/jars/rapids-4-spark_2.12-25.02.0.jar") \
    .config("spark.plugins", "com.nvidia.spark.SQLPlugin") \
    .config("spark.rapids.sql.enabled", "true") \
    .config("spark.executor.resource.gpu.amount", "1") \
    .config("spark.task.resource.gpu.amount", "1") \
    .getOrCreate()

print("RAPIDS enabled:", spark.conf.get("spark.rapids.sql.enabled", "Not set"))


25/03/13 00:01:32 WARN Utils: Your hostname, oem resolves to a loopback address: 127.0.1.1; using 192.168.5.144 instead (on interface wlp9s0)
25/03/13 00:01:32 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
25/03/13 00:01:32 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/13 00:01:33 WARN RapidsPluginUtils: RAPIDS Accelerator 25.02.0 using cudf 25.02.0, private revision 766b22becc975a64be0c2163f6fa0544a581e542
25/03/13 00:01:33 WARN RapidsPluginUtils: RAPIDS Accelerator is enabled, to disable GPU support set `spark.rapids.sql.enabled` to false.
25/03/13 00:01:33 WARN RapidsPluginUtils: spark.rapids.sql.explain is set to `NOT_ON_GPU`. Set it to 'NONE' to suppress the diagnostics logging about the query placement on the GPU.
25/03/13 00:01:33 WARN Resource

Py4JError: An error occurred while calling None.org.apache.spark.api.java.JavaSparkContext

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/home/oem/Documents/github/yelp_revisited/ylpenv/lib/python3.12/site-packages/py4j/clientserver.py", line 516, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/oem/Documents/github/yelp_revisited/ylpenv/lib/python3.12/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/oem/Documents/github/yelp_revisited/ylpenv/lib/python3.12/site-packages/py4j/clientserver.py", line 539, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while sending or receiving


#### 2. Import JSON Tables

In [ ]:
# Load each Parquet file into a Spark DataFrame
business_df = spark.read.parquet(os.path.join(parquet_dir, 'business.parquet')).sample(False, 0.1, seed=42)
# review_df   = spark.read.parquet(os.path.join(parquet_dir, 'review.parquet')).sample(False, 0.1, seed=42)
# checkin_df  = spark.read.parquet(os.path.join(parquet_dir, 'checkin.parquet')).sample(False, 0.1, seed=42)
# tip_df      = spark.read.parquet(os.path.join(parquet_dir, 'tip.parquet')).sample(False, 0.1, seed=42)
# user_df     = spark.read.parquet(os.path.join(parquet_dir, 'user.parquet')).sample(False, 0.1, seed=42)

# Show a sample from one DataFrame
business_df.show(5)

In [ ]:
spark.stop()